# Sample Integrated Model with Domino Model Monitoring

This is an example notebook to set up integrated Domino Model Monitoring of models hosted as Domino Model APIs.

## Background

Integrated model monitoring is intended to be used when the model itself deployed as a model API within your Domino cluster. This notebook walks through:

(1) Registering the training data snapshot used for training the model as a TrainingDataset.

(2) Adding in the Prediction Capture Client to your model, and deploying the model as a Domino Model API. This allows Domino to automatically capture the scoring data & model predictions for you.

(3) Setting up integrated Domino Model Monitoring for your Model API. 

(4) (Optional) Attaching a Domino Model Monitoring datasource for ingesting ground truth labels. To automate this step, the notebook walks through setting up a scheduled job to send ground truth labels to the datasource so that Domnino can monitor the model's accuracy over time.

### Step 1: Create and Register the Training Dataset

In the integrated model scenario, we'll assume the model is trained in Domino's Workbench, meaning the model's training data was brought into a Domino run. With integrated model monitoring, rather than uploading the training data set to an external data source, we can register and version it as a **TrainingSet**in Domino, and automatically ingest it when the integrated model is registered.

A **TrainingSet** is a versioned set of data, column information, and other metadata. See documentation here:

https://docs.dominodatalab.com/en/latest/api_guide/440de9/trainingsets-use-cases/

To register a training dataset, we'll import the Domino training sets client, set the training set metadata, and store a version in Domino.

In [5]:
from domino_data.training_sets import client, model
import pandas as pd
import os
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

user_name = os.environ['DOMINO_USER_NAME']

# Load the original Iris training dataset, and split into train and test sets
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data["data"], data["target"], test_size=0.2
)

# Create the training dataframe
target_column_name = "variety"

training_df = pd.DataFrame(data = X_train, columns = data.feature_names)
training_df[target_column_name] = [data.target_names[y] for y in y_train]

# Create the training set version to store this snapshot.
tsv = client.create_training_set_version(
    training_set_name="iris_python_multi_classification_{}".format(os.environ.get('DOMINO_PROJECT_NAME')),
    df=training_df,
    key_columns=[],
    target_columns=[target_column_name],
    exclude_columns=[],
    meta={"experiment_id": "0.1"},
    monitoring_meta=model.MonitoringMeta(**{
        "categorical_columns": [target_column_name],
        "timestamp_columns": [],
        "ordinal_columns": []
    })
)

print(f"TrainingSetVersion {tsv.training_set_name}:{tsv.number}")

# Save the training data locally for reference
training_df.to_csv("data/iris_train_data.csv", index_label=False)

TrainingSetVersion iris_python_multi_classification_monitor_workshop:2


### Step 2: Train the Model

Since this example uses a Domino-hosted model API, we'll start by training a simple model to deploy in Domino.

For integrated model monitoring, we train the model the same way we would any other machine learning model. However, when we create the model class that the model API will ultimately call, we need to include the Domino **DataCaptureClient**, which automatically captures the scoring data and model predictions.

To configure the **DataCaptureClient** for this model, we need to pass it the names of the features we want to monitor, as well as the name of the model prediction (or target) column, to capture the model's predictions.

The DataCaptureClient is documented here:

https://docs.dominodatalab.com/en/latest/user_guide/93e5c0/set-up-prediction-capture/

In [13]:
from xgboost import XGBClassifier
from domino_data_capture.data_capture_client import DataCaptureClient
from sklearn.metrics import accuracy_score
import uuid
import datetime
import pickle
import mlflow

# Initiate MLFlow client
client = mlflow.tracking.MlflowClient()

# Verify MLFLow URI
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])

# Create an XGBoost model
xgb_classifier = XGBClassifier(
    n_estimators=10,
    max_depth=3,
    learning_rate=1,
    objective="binary:logistic",
    random_state=123,
)

# Train the model
xgb_classifier.fit(X_train, y_train)

# Optional, save the serialized model locally 
# file_name = "models/xgb_iris.pkl"
# pickle.dump(xgb_classifier, open(file_name, "wb"))

# Set up the DataCaptureClient. Pass feature names and the target column name.
data_capture_client = DataCaptureClient(data.feature_names, [target_column_name])

# Create a model Class to call that includes the DataCaptureClient.
class IrisModel(mlflow.pyfunc.PythonModel):
    def __init__(self,model):
        self.model = model
    
    def predict(self, context, model_input, params=None):
        event_time = datetime.datetime.now(datetime.timezone.utc).isoformat()
        prediction = self.model.predict(model_input)
        
        for i in range(len(prediction)):
            # Record eventID and current time
            event_id = uuid.uuid4()
            # Convert np types to python builtin type to allow JSON serialization by prediction capture library
            model_input_value = [float(x) for x in model_input[i]]
            prediction_value = [data.target_names[prediction[i]]]
            
            # Capture this prediction event so Domino can keep track
            data_capture_client.capturePrediction(model_input_value, prediction_value, event_id=event_id,
                                timestamp=event_time)
        return prediction

model = IrisModel(xgb_classifier)

model.model

y_pred = xgb_classifier.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)

MLFLOW_TRACKING_URI: http://127.0.0.1:8765


### Step 3: Register your Model in the Model Registry

Before setting up the Model API, Domino recommends registering the new model in the Model Registry. The Model Registry tracks and manages all your machine learning models, providing documentation about how, when and where the model was created. In addition, the Model Registry allows collaborators to:

- Discover models in project-scoped and deployment-scoped registries.

- Record model metadata and lineage for auditability and reproducibility.

- Create custom model cards to capture notes on fairness, bias, and other important information.

- Manage model versions and deploy models to Domino-hosted or externally-hosted endpoints.

This context will be useful once the model is being monitored, to help determine sources of drift or explain changes in accuracy detected by Domino Model Monitoring.

See Model Registry documentation here:

https://docs.dominodatalab.com/en/latest/user_guide/3b6ae5/manage-models-with-model-registry/

In [14]:
run_timestamp = datetime.datetime.today().strftime('%Y-%m-%d')

mlflow.set_experiment(experiment_name=os.environ.get('DOMINO_PROJECT_NAME') + " " + os.environ.get('DOMINO_STARTING_USERNAME'))

with mlflow.start_run() as run:
    mlflow.log_param('n_estimators', 10)
    mlflow.log_param('max_depth', 3)
    mlflow.log_param('learning_rate', 1)
    mlflow.log_param('objective', "binary:logistic")
    mlflow.log_param('random_state', 123)
    mlflow.log_metric('accuracy', accuracy)
    model_info = mlflow.pyfunc.log_model(
        registered_model_name="DMM-Quickstart-Model-{}-{}".format(user_name, run_timestamp),
        python_model=model,
        artifact_path="test-model"
    )
print(model_info)

/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'DMM-Quickstart-Model-bryan_prosser-2024-09-19' already exists. Creating a new version of this model...
2024/09/19 09:59:10 INFO mlflow.store.mode

Created version '4' of model 'DMM-Quickstart-Model-bryan_prosser-2024-09-19'.


### Step 4: Create Model API from the Model Card

Once your model has been registered:

1) Navigate to the model registry, open the Model Card for "DMM-Quickstart-Model-DATE" (or whatever you called your model)

2) Create a new Model API with the name "DMM-Quickstart-YOURNAME" replacing your name as appropriate 

3) For Model API Source, select "Choose Model From Model Registry" and select "DMM-Quickstart-Model"

4) Once the Model API is green and says "Running", navigate to the "Configure Model Monitoring" tab in the Model API. On the right, click "Configure Monitoring", and follow the instructions. Select your training set created above as the model baseline for drift, and set the model type to Classification.

5) Score some data, using the sample Python code below. Be sure to update your URL and auth token to point to your Model API. A sample specific to your model is available in the Model API Overview tab. Domino Prediction Data Capture will capture these predictions in the back end.

![alt text](readme_images/API_Request_Python.png)

5) Wait for a bit. If you navigate to Domino Model Monitoring, the new model will appear. If you click into your new monitored model, under "Overview" in the "Ingest History" tab, the training data should be shown as ingested and "Done". However, under "Data Drift", your model will still say "No Prediction Data Added" for about an hour. The Model API Monitoring tab will say "Waiting for Prediction Data." The prediction data from step 4 has been captured, but you have to wait for the first automated ingest for that drift data to appear in the Model Monitoring UI and to move to the next steps.

6) Once data drift ingestion has happened, a new Domino Dataset called "prediction_data" will appear in your Project Domino Datasets list, and the Model Monitoring Data Drift section will populate.

### Save your model API URL, model API auth token, and DMM Model ID to the Project config file. 

To avoid saving the model API url & auth token in a git repo, we can save them to the config file directly in the Workbench.

1) Navigate back to the Workbench, open the Artifacts section and open the 'DMM_config.yaml' file.
2) Click "Edit", and copy and paste your new model url and auth token into the  "integrated_model_url" and "integrated_model_auth" fields.
3) Navigate to Domino Model Monitoring. Click into your new model, copy the Model ID on the right, and save it to the 'integrated_model_id' field.
4) Save the config file.
5) Navigate back to your Workspace. In the File Changes menu on the left, pull all latest changes. Now the config file in this Workspace is in sync with the Project files.

In [17]:
# Test your new model API by sending some scoring data

import yaml
import requests

# Load the config file
with open("/mnt/artifacts/DMM_config.yaml") as yamlfile:
    config = yaml.safe_load(yamlfile)

 
response = requests.post(config['integrated_model_url'], 
    auth=(
            config['integrated_model_auth'],
            config['integrated_model_auth'] 
    ),
    json={
       "data":  [  [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [6.7, 3.3, 5.7, 2.5],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [6.7, 3.3, 5.7, 2.5],
        [6.7, 3. , 5.2, 2.3],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [6.7, 3.3, 5.7, 2.5],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [6.7, 3.3, 5.7, 2.5],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [6.7, 3.3, 5.7, 2.5],]
    }
)
 
print(response.status_code)
print(response.headers)
print(response.json())

200
{'Date': 'Thu, 19 Sep 2024 11:08:38 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-Request-ID': '7EHOTK2VNAYZ1LPM', 'Domino-Server': 'nginx-ingress,model-api,', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'POST', 'Access-Control-Allow-Headers': 'authorization,content-type', 'Content-Security-Policy': "frame-ancestors 'self' demo2.dominodatalab.com; ", 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'SAMEORIGIN always', 'Content-Encoding': 'gzip'}
{'model_time_in_ms': 8, 'release': {'harness_version': '0.1', 'registered_model_name': 'DMM-Quickstart-Model-bryan_prosser-2024-09-19', 'registered_model_version': '4'}, 'request_id': '7EHOTK2VNAYZ1LPM', 'result': [0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2], 'timing': 8.363962173461914}


### Step 5 (Optional): Register a Ground Truth Dataset

Typically for this step you would fetch actual ground truth data (the actual outcomes from what your model predicted on), 
join the actual outcomes with your prediction data, and upload into a datasource attached to model monitoring for Model Quality 
analysis.

However, for purposes of creating a quick demo, we'll make up some fake ground truth data using the model predictions captured with Domino's
data capture client. These predictions are stored in an automatically-generated Domino Dataset called "prediction_data"

Once Data has ingested (roughly one hour), a "prediction_data" Domino Dataset will be added to the Project.

1) Navigate to the Domino Dataset Folder on the left (back from /mnt/ , then "data/prediction_data/...")
Copy the path to read in your registered model predictions.

2) Join the Predictions to make your ground truth dataset, shuffle some labels to simulate classification errors, and save the ground truth csv

3) Upload the csv to the s3 bucket attached as a Domino Model Monitoring Dataset


#### Step 5.1 Connect an external data source to this Project and to Domino Model Monitoring

Integrated models do not capture ground truth labels for you, since they are generally captured after the fact. Domino requires an external data source to ingest these ground truth labels.

This example will use the same Monitoring Datasource set up in "1_Initial_Setup.ipynb", use the data source names saved in the config file.

#### Step 5.2 Create a "dummy" ground truth dataset and upload it to the external datasource

Typically Ground Truth Data would be captured somewhere external to Domino, then uploaded to your Monitoring Datasource. A call to the DMM API can alert DMM that new ground truth data is available for ingestion.

However, in this example, we will have to create our own "dummy" ground truth data, using the scoring and prediction data captured but the DataCaptureClient set up in Step 2. 

The DataCaptureClient automatically saves scoring and prediction data in a parquet file in a new Domino Dataset in this Project called "prediction_data". 

1) Ensure the first batch of predictions have been ingested into your new model. A new Domino Dataset called "prediction_data" should be created and populated with a folder containing the initial prediction data captured.
3) In this Workspace, navigate to the "prediction_data" Domino dataset (under "/mnt/data/prediction_data"), copy the file path to one of the parquet files in there.

The path should be formatted like this:

"/mnt/data/prediction_data/{PREDICTION_DATA_ID}/{DATE}/{TIME}/predictions_{ID}.parquet"

Paste in the cell below, and take a look at the captured prediction data and predictions.

In [3]:
import pandas as pd
import yaml

# UPDATE this PATH
path = '/mnt/data/prediction_data/663c03517e57664fd44c86fd/$$date$$=2024-05-08Z/$$hour$$=23Z/predictions_68a8751b-14b0-450e-954f-f3e3fbbc0c0f.parquet'

predictions = pd.read_parquet(path)

# Save the prediction_data_dir ID to the config file
with open("/mnt/artifacts/DMM_config.yaml") as yamlfile:
    config = yaml.safe_load(yamlfile)

config['prediction_data_dir'] = path.split('/')[4]

with open("/mnt/artifacts/DMM_config.yaml", "w") as yamlfile:
    config = yaml.dump(
        config, stream=yamlfile, default_flow_style=False, sort_keys=False
    )
    
print(predictions.shape)
predictions.head()

(34, 8)


,petal length (cm),petal width (cm),sepal length (cm),sepal width (cm),variety,timestamp,__domino_timestamp,event_id
0,1.1,0.1,4.3,3.0,setosa,2024-05-08 23:03:09.002378+00:00,2024-05-08T23:03:09.011965+00:00,4f2507dc-1793-44fb-bbc4-5beba619aabe
1,1.2,0.2,5.8,4.0,setosa,2024-05-08 23:03:09.002378+00:00,2024-05-08T23:03:09.012558+00:00,32557c22-b160-4564-952e-1f6ffe735412
2,1.5,0.4,5.7,4.4,setosa,2024-05-08 23:03:09.002378+00:00,2024-05-08T23:03:09.012842+00:00,e227effe-dc4a-46eb-972c-cbefb7ea8c41
3,5.7,2.5,6.7,3.3,virginica,2024-05-08 23:03:09.002378+00:00,2024-05-08T23:03:09.013037+00:00,4cff8485-dca7-45ef-ac13-0649191cf43f
4,1.2,0.2,5.8,4.0,setosa,2024-05-08 23:03:09.002378+00:00,2024-05-08T23:03:09.013259+00:00,30d0a916-5779-4e4a-922a-de189ec01f19


The Ground Truth dataset needs 2 columns: 

1) The existing event ID column from the model predictions.
   
    This column has the join keys for joining ground truth labels to your model's predictions

3) Your new column containing ground truth labels.


In [5]:
import datetime

event_id = predictions['event_id']
iris_ground_truth = predictions['variety']

# Create a new dataframe
ground_truth = pd.DataFrame(columns=['event_id', 'iris_ground_truth'])
ground_truth['event_id'] = event_id
ground_truth['iris_ground_truth'] = iris_ground_truth

# These row labels help find some diferent iris types in our initial scoring data
end_index = predictions.shape[0]
mid_index = int(round(predictions.shape[0] / 2, 0))

# Simulate some classifcation errors. This makes our confusion matrix interesting.
ground_truth.iloc[0, 1] = 'virginica'
ground_truth.iloc[1, 1] = 'versicolor'
ground_truth.iloc[mid_index-1, 1] = 'versicolor'
ground_truth.iloc[mid_index, 1] = 'virginica'
ground_truth.iloc[end_index-2, 1] = 'setosa'
ground_truth.iloc[end_index-1, 1] = 'setosa'

# Save this example ground truth csv to your file to your Project files for reference.

date = datetime.datetime.today()
month = date.month
day = date.day
year = date.year

date = str(datetime.datetime.today()).split()[0]

ground_truth.to_csv('data/{}_iris_ground_truth_{}_{}_{}.csv'.format(user_name, month, day, year), index=False)

#### Step 5.3 Upload the ground truth file to a Domino Model Monitoring data source.

Ground truth labels must come from an external data source attached to Domino Model Monitoring. The Model API does not capture ground truth labels, since they typically become available after the prediction.

The AWS example uses a Domino Data Source, you could also use boto3 or other methods to upload data to s3.

The Azure example uses a Domino Data Source with ADLS.

#### AWS: s3

In [6]:
# For this approach, add an s3 Domino Data Source bucket to your Project. Then, copy the first fe linwes of the automatically generated Python code.
from domino.data_sources import DataSourceClient
import yaml

# Load the config file
with open("/mnt/artifacts/DMM_config.yaml") as yamlfile:
    config = yaml.safe_load(yamlfile)

# instantiate a client and fetch the datasource instance
object_store = DataSourceClient().get_datasource(config['workbench_datasource_name']) 

# list objects available in the datasource
objects = object_store.list_objects()

object_store.upload_file("{}_iris_ground_truth_{}_{}_{}.csv".format(user_name, month, day, year), "data/{}_iris_ground_truth_{}_{}_{}.csv".format(user_name, month, day, year))

#### Azure: ADLS

In [ ]:
# from domino.data_sources import DataSourceClient

# # instantiate a client and fetch the datasource instance
# object_store = DataSourceClient().get_datasource("adlsdatasource")

# # list objects available in the datasource
# objects = object_store.list_objects()

# object_store.upload_file("iris_ground_truth_{}_{}_{}.csv".format(month, day, year), "data/iris_ground_truth_{}_{}_{}.csv".format(month, day, year))

#### Step 5.4 First Time Registration of Ground Truth Labels via the API

The final step is to register Ground Truth Labels with Domino Model Monitoring.

This can be done in the Model Monitoring UI using the Ground Truth Config file, or using the Domino Model Monitoring API.

Documentation here: https://docs.dominodatalab.com/en/latest/api_guide/f31cde/model-monitoring-api-reference/#_registerDatasetConfig

You’ll need the following:

1) The Monitoring Dataset & config file set up in "1_Initial_Setup.ipynb". Make sure you have synced the workspace since Step 4 to add your DMM model ID to the config file.
    
2) The column name of your new, ground truth labels 

3) Your original target (or prediction) column name

In [7]:
import json
import os
import requests
import datetime
import yaml

# The name of the file uploaded to s3 above
gt_file_name = "{}_iris_ground_truth_{}_{}_{}.csv".format(user_name, month, day, year)

# The name of the column containing ground truth labels
GT_column_name = 'iris_ground_truth'

# Your original target column name
target_column_name = 'variety'

# Load the config file
with open("/mnt/artifacts/DMM_config.yaml") as yamlfile:
    config = yaml.safe_load(yamlfile)


ground_truth_url = "https://{}/model-monitor/v2/api/model/{}/register-dataset/ground_truth".format(config['url'], config['integrated_model_id'])

print('Registering {} From S3 Bucket in DMM'.format(gt_file_name))
 
# create GT payload    
 
# Set up call headers
headers = {
           'X-Domino-Api-Key': os.environ['DOMINO_USER_API_KEY'],
           'Content-Type': 'application/json'
          }

 
ground_truth_payload = """
{{
    "variables": [{{
    
            "valueType": "categorical",
            "variableType": "ground_truth",
            "name": "{2}", 
            "forPredictionOutput": "{3}"
        
    }}],
    "datasetDetails": {{
            "name": "{0}",
            "datasetType": "file",
            "datasetConfig": {{
                "path": "{0}",
                "fileFormat": "csv"
            }},
            "datasourceName": "{1}",
            "datasourceType": "{4}"
        }}
}}
""".format(gt_file_name, config['DMM_datasource_name'], GT_column_name, target_column_name, config['datasource']['type'])
 
# Make api call
ground_truth_response = requests.request("PUT", ground_truth_url, headers=headers, data = ground_truth_payload)
 
# Print response
print(ground_truth_response.text.encode('utf8'))
 
print('DONE!')


Registering iris_ground_truth_5_9_2024.csv From S3 Bucket in DMM
b''
DONE!


### Next Steps

Going forward, Domino will automatically capture all prediction data going across your Model API. It will ingest these predictions for Drift detection once per day. You can set a schedule to determine when this ingest happens.

To periodically upload ground truth labels, repeat the previous step, but without the “variables” in the ground truth payload (this only needs to be done once). As new ground truth labels are added, point Domino to the path to the new labels in the monitoring data source by pinging the same Model Monitoring API:

ground_truth_payload = """

{{

       "datasetDetails": {{
        
            "name": "{0}",
            "datasetType": "file",
            "datasetConfig": {{
                "path": "{0}",
                "fileFormat": "csv"
            }},
            "datasourceName": "{1}",
            "datasourceType": "s3"
        }}
}}""".format(gt_file_name, data_source, GT_column_name, target_column_name)



### Automation with Domino Jobs
To simulate Domino Model Monitoring over time, you can try out running the following two scripts as scheduled Domino Jobs:

Scripts are in the "integrated_model_scripts" directory.

**(1) daily_scoring.py**

Daily scoring simulates sending data to the model API for scoring. Data is read in, sent to the Domino Model API, and predictions are returned. Domino's Prediction Capture Client captures the scoring data and model predictions. Every 24 hours, the captured data is ingested into the Drift Monitoring dashboard. Note that while this example uses a batch job, integrated model APIs capture both batch and real time data sent to the API.

**(2) daily_ground_truth.py**

Daily ground truth simulates uploading actual outcomes after the predictions have been made. A scheduled Domino Job writes the latest ground truth labels to an s3 bucket, then calls the Domino Model Monitoring API with the path to the file with the latest ground truth labels.

**Important**
If you schedule these two jobs, be sure that daily_ground_truth.py runs after both daily_scoring.py and the scheduled drift check in DMM.

Suggested schedule:

daily_scoring.py - scheduled Domino Job at 1am

Data Drift check in DMM - scheduled for 2am

daily_ground_truth.py - scheduled Domino Job at 3am